In [25]:
import torch
from torch import nn

torch.__version__

'2.0.0+cu117'

In [26]:
from torchvision import transforms

batch_size = 32
img_size = 224

train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

test_transform = transforms.Compose([
    # transforms.Resize(230),
    # transforms.CenterCrop(img_size),
    # transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    transforms.ToTensor()
    ])

In [27]:
from torchvision import datasets
from torch.utils.data import DataLoader

train_set = datasets.ImageFolder("./Dataset/train", transform=train_transform)
trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

test_set = datasets.ImageFolder("./Dataset/test", transform=test_transform)
testloader = DataLoader(test_set, shuffle=True)

In [28]:
feature, target = next(iter(trainloader))
feature.shape

torch.Size([32, 3, 224, 224])

In [29]:
import torchvision

device = 'cuda'
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights 
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

In [30]:
for param in model.features.parameters():
    param.requires_grad = False

In [31]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=1280, 
                    out_features=10, # same number of output units as our number of classes
                    bias=True)).to(device)

In [32]:
loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

In [33]:
from timeit import default_timer as timer 
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format). 
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [34]:
def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.
    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.
    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [36]:
from tqdm.auto import tqdm
from timeit import default_timer as timer 
import gc
# Set the seed and start the timer
torch.manual_seed(42)
train_time_start_on_cpu = timer()

# Set the number of epochs (we'll keep this small for faster training times)
epochs = 25

# Create training and testing loop
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n-------")
    ### Training
    train_loss = 0
    # Add a loop to loop through training batches
    for batch, (X, y) in enumerate(trainloader):
        model.train() 
        # 1. Forward pass
        y_pred = model(X.to(device))

        # 2. Calculate loss (per batch)
        loss = loss_fn(y_pred.to(device), y.to(device))
        train_loss += loss # accumulatively add up the loss per epoch 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Print out how many samples have been seen
        if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/{len(trainloader.dataset)} samples")

    # Divide total train loss by length of train dataloader (average loss per batch per epoch)
    train_loss /= len(trainloader)
    
    ### Testing
    # Setup variables for accumulatively adding up loss and accuracy 
    test_loss, test_acc = 0, 0 
    model.eval()
    with torch.inference_mode():
        for X, y in testloader:
            # 1. Forward pass
            test_pred = model(X.to(device))
           
            # 2. Calculate loss (accumatively)
            test_loss += loss_fn(test_pred.to(device), y.to(device)) # accumulatively add up the loss per epoch

            # 3. Calculate accuracy (preds need to be same as y_true)
            test_acc += accuracy_fn(y_true=y.to(device), y_pred=test_pred.argmax(dim=1).to(device))
        
        # Calculations on test metrics need to happen inside torch.inference_mode()
        # Divide total test loss by length of test dataloader (per batch)
        test_loss /= len(testloader)

        # Divide total accuracy by length of test dataloader (per batch)
        test_acc /= len(testloader)
        torch.cuda.empty_cache()
        gc.collect()
    ## Print out what's happening
    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n")

# Calculate training time      
train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu, 
                                           end=train_time_end_on_cpu,
                                           device=str(next(model.parameters()).device))

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch: 0
-------
Looked at 0/1310 samples

Train loss: 1.28418 | Test loss: 2.13294, Test acc: 33.96%

Epoch: 1
-------
Looked at 0/1310 samples

Train loss: 1.26823 | Test loss: 2.07986, Test acc: 37.74%

Epoch: 2
-------
Looked at 0/1310 samples

Train loss: 1.25858 | Test loss: 2.02662, Test acc: 42.32%

Epoch: 3
-------
Looked at 0/1310 samples

Train loss: 1.22975 | Test loss: 2.06279, Test acc: 38.01%

Epoch: 4
-------
Looked at 0/1310 samples

Train loss: 1.20196 | Test loss: 2.04156, Test acc: 39.35%

Epoch: 5
-------
Looked at 0/1310 samples

Train loss: 1.20132 | Test loss: 1.99216, Test acc: 43.13%

Epoch: 6
-------
Looked at 0/1310 samples

Train loss: 1.18644 | Test loss: 2.03724, Test acc: 39.08%

Epoch: 7
-------
Looked at 0/1310 samples

Train loss: 1.17727 | Test loss: 2.02926, Test acc: 40.43%

Epoch: 8
-------
Looked at 0/1310 samples

Train loss: 1.15958 | Test loss: 1.98194, Test acc: 42.32%

Epoch: 9
-------
Looked at 0/1310 samples

Train loss: 1.16640 | Test los